# Trouver une bonne stratégie au 421

L'objectif est d'apprendre à jouer au jeu du [421](https://fr.wikipedia.org/wiki/421_(jeu)) contre un adversaire. 

Il y a 21 pions au départ, répartis aléatoirement : c'est la *charge*. <br>Pour la *décharge*, on établit 2 stratégies *déterministes* : 
* l'une en tant que meneur (vous avez la main et décidez quand vous arrêter), 
* l'autre en tant qu'opposant (vous jouez en fonction de la combinaison et du nombre de coups du meneur).


## Init

In [37]:
import numpy as np
import matplotlib.pyplot as plt

In [38]:
total_budget = 21

In [39]:
dice = np.arange(1,7)

In [40]:
throws = np.arange(3)

In [41]:
def get_scores():
    scores = {(4,2,1): 11, (1,1,1): 7, (2,2,1): 0}
    scores.update({(d,1,1): d for d in dice if d > 1})
    scores.update({(d,d,d): d for d in dice if d > 1})
    scores.update({(d,d - 1,d - 2): 2 for d in dice if d > 2})
    specials = list(scores.keys())
    scores.update({(d,e,f): 1 for d in dice 
                  for e in dice if e <= d 
                  for f in dice if f <= e and (d,e,f) not in specials})
    return scores, specials

In [42]:
scores, specials = get_scores()

In [43]:
reverse_dice = np.arange(6, 0, -1)

In [44]:
def get_rank():
    rank = {(4,2,1): 0, (1,1,1): 1}
    rank.update({(d,1,1): 2 + i for i,d in enumerate(reverse_dice) if d > 1}) 
    rank.update({(d,d,d): 7 + i for i,d in enumerate(reverse_dice) if d > 1})
    rank.update({(d,d - 1,d - 2): 12 + i for i,d in enumerate(reverse_dice) if d > 2})
    specials = list(rank.keys())    
    i = 16
    for d in reverse_dice:
        for e in range(d, 0, -1):
            for f in range(e, 0, -1):
                if (d,e,f) not in specials:
                    rank[(d,e,f)] = i
                    i += 1
    return rank

In [45]:
rank = get_rank()

In [46]:
states = list(scores.keys())

In [47]:
actions = [(a,b,c) for a in range(2) for b in range(2) for c in range(2)]

## Training

Value iteration

In [48]:
def transition_prob(state, action):
    prob = {}
    nb = np.sum(action)
    for i in range(6**nb):
        new_state = np.array(state)
        die = i
        for j in range(3):
            if action[j]:
                new_state[j] = die % 6 + 1
                die = die // 6
        new_state = tuple(sorted(new_state,reverse = True))
        if new_state in prob:
            prob[new_state] += 1 / 6**nb
        else:
            prob[new_state] = 1 / 6**nb
    return prob

In [49]:
def get_score(state1, state2):
    if rank[state1] < rank[state2]:
        # player 1 wins
        return scores[state1]
    elif rank[state1] > rank[state2]:
        # player 2 wins
        return -scores[state2]
    else:
        # random tie breaking (to be taken into account)
        return 0    

In [50]:
def opponent_training():
    V = {}    
    policy = {}
    for throw in throws:
        if throw == 0:
            for state in states:
                for lead_state in states:
                    V[(lead_state, state, throw)] = get_score(state, lead_state)
        else:      
            for state in states:
                for lead_state in states:
                    max_value = np.NINF
                    for action in actions:
                        value = 0
                        prob = transition_prob(state, action)
                        for s in prob:
                            value += prob[s] * V[(lead_state, s, throw - 1)]
                        if value > max_value:
                            max_value = value
                            best_action = action
                    V[(lead_state, state, throw)] = max_value
                    if best_action != (0,0,0):
                        policy[(lead_state, state, throw)] = best_action
    return V, policy

In [51]:
V, opponent = opponent_training()

In [52]:
def lead_training(opponent_value):
    V = {}
    for throw in throws:
        for lead_state in states:
            value = 0
            prob = transition_prob((1,1,1), (1,1,1))
            for s in prob:
                value += prob[s] * opponent_value[(lead_state, s, throw)]
            V[(lead_state, throw)] = -value
    policy = {}
    for throw in throws:
        if throw > 0:
            for lead_state in states:
                max_value = np.NINF
                for action in actions:
                    value = 0
                    prob = transition_prob(lead_state, action)
                    for s in prob:
                        value += prob[s] * V[(s, throw - 1)]
                    if value > max_value:
                        max_value = value
                        best_action = action
                if best_action != (0,0,0):
                    policy[(lead_state, throw)] = best_action
    return policy

In [53]:
lead = lead_training(V)

## Save policy

In [54]:
import csv

In [55]:
def save_lead(lead, filename = 'lead.csv'):
    with open(filename, mode='w') as csvfile:
        writer = csv.writer(csvfile)
        for (state, throw) in lead:
            row = [state[i] for i in range(3)]
            row += [throw]
            action = lead[(state, throw)]
            row += [action[i] for i in range(3)]
            writer.writerow(row)

In [56]:
save_lead(lead)

In [57]:
def save_opponent(opponent, filename = 'opponent.csv'):
    with open(filename, mode='w') as csvfile:
        writer = csv.writer(csvfile)
        for (lead_state, state, throw) in opponent:
            row = [lead_state[i] for i in range(3)]
            row += [state[i] for i in range(3)]
            row += [throw]
            action = opponent[(lead_state, state, throw)]
            row += [action[i] for i in range(3)]
            writer.writerow(row)

In [58]:
save_opponent(opponent)

## Play the game

In [59]:
def random_state():
    a = np.random.choice(6) + 1
    b = np.random.choice(6) + 1
    c = np.random.choice(6) + 1
    return tuple(sorted((a,b,c),reverse = True))

In [60]:
def transition_prob(state, action):
    prob = {}
    nb = np.sum(action)
    for i in range(6**nb):
        new_state = np.array(state)
        die = i
        for j in range(3):
            if action[j]:
                new_state[j] = die % 6 + 1
                die = die // 6
        new_state = tuple(sorted(new_state, reverse = True))
        if new_state in prob:
            prob[new_state] += 1 / 6**nb
        else:
            prob[new_state] = 1 / 6**nb
    return prob

In [61]:
def move(state, action):
    prob = transition_prob(state, action)
    i = np.random.choice(np.arange(len(prob)), p = list(prob.values()))
    return list(prob.keys())[i]

In [62]:
# Score of the leader (single round)

def lead_score(lead, opponent, lead_budget):
    # leader
    state = random_state()
    throw = 2
    #print(state, throw)
    while throw > 0 and (state, throw) in lead and lead[(state, throw)] != (0,0,0):
        action = lead[(state, throw)]
        state = move(state, action)
        throw -= 1
        #print(state, throw)
    lead_state = state
    # opponent
    state = random_state()
    throw = 2 - throw
    #print(state, throw)
    while throw > 0 and (lead_state, state, throw) in opponent and opponent[(lead_state, state, throw)] != (0,0,0):
        #print(state)
        action = opponent[(lead_state, state, throw)]
        state = move(state, action)
        throw -= 1
        #print(state, throw)
    return get_score(lead_state, state)

In [63]:
# Winner of a game (returns the winner)

def game(lead1, opponent1, lead2, opponent2):
    # charge
    budget = total_budget
    budget1 = 0
    budget2 = 0
    while budget > 0:
        state1 = random_state()
        state2 = random_state()
        result =  get_score(state1, state2)
        cost = min(abs(result), budget)
        if result > 0:
            budget2 += cost
            leader = 2
        else:
            budget1 += cost
            leader = 1
        budget -= cost
    # decharge
    while budget1 > 0 and budget2 > 0:
        if leader == 1:
            result = lead_score(lead1, opponent2, budget1)
            budget1 -= result
            budget2 += result
            if result > 0:
                leader = 2
        else:
            result = lead_score(lead2, opponent1, budget2)
            budget1 += result
            budget2 -= result
            if result > 0:
                leader = 1          
    if budget1 > 0:
        return 2
    else:
        return 1

In [64]:
lead1 = lead
opponent1 = opponent
lead2 = {}
opponent2 = {}

In [65]:
# Basic strategy

def keep_ones(state):
    return tuple([int(state[i] != 1) for i in range(3)])

In [66]:
# Keep only special combinations

def lead_keep_specials():
    lead = {}
    for state in states:
        if state not in specials:
            for throw in throws:
                if throw > 0:
                    lead[(state,throw)] = keep_ones(state)
    return lead

In [67]:
# Opponent

def opponent_keep_ones():
    opponent = {}
    for lead_state in states:
        for state in states:
            if get_score(state, lead_state) <= 0:
                for throw in throws:
                    if throw > 0:
                        opponent[(lead_state,state,throw)] = keep_ones(state)
    return opponent

In [68]:
lead2 = lead_keep_specials()

In [69]:
opponent2 = opponent_keep_ones()

In [70]:
# One game
game(lead1, opponent1, lead2, opponent2)

2

In [71]:
# Multiple games
n_games = 100
victory = 0
for i in range(n_games):
    if game(lead1, opponent1, lead2, opponent2) == 1:
        victory += 1
victory

71